Hello Flatlanders!
===
Or soon to be flatlanders at least!

In this set of notebooks I'm going to play around with uses for dimensionality reduction 
I like dimesnionality recution for many reasons, and it's rather useful in program and data analysis. As with all the notebooks here this is just for hobby and toy examples to help me learn the concepts and to offer me a set of examples to go back to when I actually need these techniques.

There are many good reasons to like dimensioality reduction, however, if I'm being honest, I like DR because it's pretty. DR is often used to help classify and organize sets of data. You can take data will million or more features and reduce it down to around 100 key components and then in that ~100D space you can craft classifiers and clusters to organize your data. And if you desire, you can go straight to 2 or 3 dimensions from you high dimensional space, or your lower (but still larger than we're visually use to) dimensional space 

And I'm tired of writing this bit, 
DR is cool. There's lots of ways to do it,
where your smaller dims, x, y, z, are some linear combination of your previous $N$ dims (which is used in many modern classifiers due to speed and memory) or where x, y, z, are some non linear combination ($x=N_{i}^{2}-3N_{j}^{\frac{1}{2}}$, $y=...$, $z=...$)

These different ways to combine your data have large impacts on how your final output is distributed, and depending on how your features are related, some methods will offer more insightful visualizations than others.

We'll get to it, I'm tired of this, Hurray I'm going to see if that latex bit worked out..
(It did but I forgot the closing ')')

Ok, Because I haven't yet gotten to play with this, I'm going to bring in my Free Book dataset. One night I drank a half pint of whiskey and (by hand so as not to violate the website rules) downloaded most of the top 1oo free ebooks from Project Gutenberg. 

I'm going to do a really cheap parsing where it's all lowercase, and only split on ' ', '\n', '\t', '\r' (Why does '\r' exist?) We're also going to implement a lantent semantic library based on my lsalib, but all the code will be hosted here because I need to review and update that library anyway, so why not! :D

In [1]:
# All imports will be done here
# Not my libraries
import os
import sys
import numpy
import math 
from sklearn.decomposition import ProjectedGradientNMF
import re

# My personal Libraries 
import kmlistfi

In [3]:
fls = kmlistfi.les('/home/keith/Documents/Databases/Datasets/GutTop100Books/')

In [4]:
len(fls)

99

In [5]:
# Ok for now this is a block copy and paste
#    but I'm tired, such is as it is
class termDocMatrix(object):
    def __init__(self, saveVerbose=True, wcThreshold=2, parseOn=" "):
        # Get term-document matrix
        # transormation/modified weighting of term-doc matrix 
        # dimensionality reduction
        # clustering of documents in reduced space
        self.wcThreshold = wcThreshold
        self.saveVerbose = saveVerbose
        self.parseOn = parseOn
        self.mD = {}
        self.tdm = []
        self.tdmraw = []
        self.termraw = []
        self.docs = []
        self.docsSize = []
        self.terms = []
        self.docCount = 0
        self.idfweight = False
        self.P = []
        self.Q = []
        self.er = []
        self.idfs = []
        return
    def add(self, newThing, docs=""):
        def _mergeDict(self, newD, docs=""):
            # Grab doc count so far
            docIndex = self.docCount
            # Get the correct title
            if docs == "":
                docs = docIndex
            self.docs.append(docs)
            # Add doc size (useful with tdmraw)
            tdWeight = float(sum(newD.values()))
            if tdWeight == 0:
                tdWeight = 1.
            self.docsSize.append(tdWeight)
            # Add newD to mD
            for key in newD:
                if key in self.mD:
                    if len(self.mD[key]) < self.docCount:
                        for i in range(len(self.mD[key]), self.docCount):
                            self.mD[key].append(0.)
                    self.mD[key].append(newD[key]/float(tdWeight))
                else:
                    if self.docCount > 0:
                        self.mD[key] = [0.]
                        for i in range(1, self.docCount):
                            self.mD[key].append(0.)
                        self.mD[key].append(newD[key]/float(tdWeight))
                    else:
                        self.mD[key] =  [newD[key]/float(tdWeight)]
            self.docCount += 1
            return 
            
        
        # Ok what was I just given?
        if type(newThing) == list:
            # Shit.. Ok we can handle this, a list of what?
            if len(newThing) > 0:
                if type(newThing[0]) == dict:
                    # Sweet, it's some dicts, merge them!
                    # Wait, what about the title var?
                    if (docs != "") and (type(docs) == list) and (len(docs) == len(newThing)):
                        for i in range(len(newThing)):
                            _mergeDict(self, newThing[i], docs[i])
                    else:
                        for i in range(len(newThing)):
                            _mergeDict(self, newThing[i])

                elif type(newThing[0]) in [float, long, int, str, unicode]:
                    # Well, I mean I don't see why numbers can't be LSA'ized
                    # Convert list to dict, then merge
                    newD = {}
                    for i in range(len(newThing)):
                        if newThing[i] in newD:
                            newD[newThing[i]] += 1
                        else:
                            newD[newThing[i]] = 1
                    _mergeDict(self, newD, docs)
                else:
                    raise(TypeError, "Elements of list are not valid inputs")

                

        elif type(newThing) == dict:
            # Woo this is simple!
            _mergeDict(self, newThing, docs)

        elif type(newThing) == str:
            # I'm assuming I'm to add this string to the term doc Matrix
            strList = newThing.split(self.parseOn)
            newD = {}
            for i in range(len(strList)):
                if strList[i].strip() in newD:
                    newD[strList[i].strip()] += 1
                else:
                    newD[strList[i].strip()] = 1
            _mergeDict(self, newD, docs)
        return

    def weight_idf(self):
        # Now we're weighting it, booyea
        # The weighting applied here is idf, and assumes td weighting was applied earlier
        #   idf: inverse document frequency: log(N/ni)
        #     if every doc has word ni, the it zeros out the row
        #     Rather than math it, check for it first
        self.idfweight = True
        for key in self.mD:
            # Saving raw state allows matrix to grow w/o redoing everything
            # only done if 
            if self.saveVerbose == True:
                self.termraw.append(key)
            # This chunk is to check for the idf weight condition:
            #   if every doc has the term, then it's not worth 'mathing'
            #   and instead can be eliminated
            idf = False
            if len(self.mD[key]) < self.docCount:
                idf = True
                for i in range(len(self.mD[key]), self.docCount):
                    self.mD[key].append(0.)
            # Saving raw matrix
            if self.saveVerbose == True:
                self.tdmraw.append(self.mD[key])
            # Scan row for a zero
            if idf == False:
                for i in range(len(self.mD[key])):
                    if self.mD[key][i] == 0:
                        idf = True
                        break
            # CURRENTLY AN ERROR DUE TO TD WEIGHTING EARLIER: FIXED
            if (len(filter(None, self.mD[key])) >= self.wcThreshold) and idf == True:
                self.terms.append(key)
                self.tdm.append(self.mD[key])
        
        # Ok now it's actually time to start weighting
        self.tdm = numpy.array(self.tdm)
        #print len(self.tdm)
        for i in range(len(self.tdm)):
            #print self.terms[i]
            ni = float(numpy.count_nonzero(self.tdm[i]))
            if ni == 0:
                raise ValueError("ARG HOW ARE THERE NO NON ZERO ELIMENTS")
            #print ni, self.docCount
            idfValue = math.log(self.docCount/ni)
            self.tdm[i] = self.tdm[i] * idfValue
            self.idfs.append(idfValue)
            
        return

    def svd(self):
        return

    def spectralEmbed(self, k, n_neighbors=10):
        from sklearn import manifold
        se = manifold.SpectralEmbedding(n_components=k, n_neighbors=n_neighbors)
        Yse = se.fit_transform(self.tdm)
        print len(Yse)
        return Yse

    def nmf(self, k):
        
        nmf = ProjectedGradientNMF(n_components=k, max_iter=200)
        P = nmf.fit_transform(self.tdm)
        Q = nmf.components_.T
        self.P = P
        self.Q = Q
        self.er = nmf.reconstruction_err_
        print "\tNMF Error: ", self.er
        return P, Q

In [18]:
tdm = termDocMatrix()
for i in range(len(fls)):
    flname = fls[i]
    fl = open(flname, 'r')
    lines = fl.read()
    fl.close()
    flname = flname.split('/')[-1][:-4]
    print '\t', i, '\t', flname
    line = re.sub('[\t\n\r]', ' ', lines)
    line = line.lower()
    line = line.split(' ')
    tdm.add(line, flname)
    
tdm.add("Goffy Null", "Baises Entry")    

	0 	HowtoAnalyzePeopleonSight
	1 	TheRomanceofLust
	2 	AStudyInScarlet
	3 	MythsandLegendsofAncientGreeceandRome
	4 	DonQuixote
	5 	HeartofDarkness
	6 	DavidCopperfield
	7 	AdvofSherlockHolmes
	8 	ThreeMeninaBoat
	9 	DrJekyllandMrHyde
	10 	AnneOfGreenGables
	11 	AChristmasCarol
	12 	SenseandSensibility
	13 	Pygmalion
	14 	TheNarrativeoftheLifeOfFrederick
	15 	Emma
	16 	Dracula
	17 	AutobiographyofBenjaminFranklin
	18 	TheLegendofSleepyHollow
	19 	LesMiserables
	20 	MySecretLifeV1
	21 	CrimeandPunishment
	22 	MaupassantOrginalShortStories
	23 	Dubliners
	24 	PrideandPrejudice
	25 	WarAndPeace
	26 	TheEssaysofMontaigneComplete
	27 	TheJungle
	28 	EnglishLiterature
	29 	HeartofDarkness2
	30 	EthanFrome
	31 	ofLeavesofGrass
	32 	DonJuan
	33 	BeyondGoodAndEvil
	34 	UncleTomsCabin
	35 	OccurrenceatOwlCreek
	36 	WutheringHeights
	37 	TheBrothersKaramazov
	38 	HardTimes
	39 	ThroughTheLookingGlass
	40 	GulliversTravels
	41 	AlicesAdventuresInWonderland
	42 	TreasureIsland
	43 	theImportanceOfB

In [19]:
# Worth a shot
tdm.weight_idf()


In [20]:
tdm.nmf(2)

/home/keith/anaconda2/lib/python2.7/site-packages/sklearn/utils/__init__.py:75: DeprecationWarning: Class ProjectedGradientNMF is deprecated; It will be removed in release 0.19. Use NMF instead.'pg' solver is still available until release 0.19.
  warnings.warn(msg, category=DeprecationWarning)
/home/keith/anaconda2/lib/python2.7/site-packages/sklearn/decomposition/nmf.py:775: DeprecationWarning: 'pg' solver will be removed in release 0.19. Use 'cd' solver instead.
  " Use 'cd' solver instead.", DeprecationWarning)


	NMF Error:  0.156002660872


(array([[  4.13811841e-03,   1.48677717e-02],
        [  2.82205777e-08,   7.42616800e-08],
        [  2.58111799e-06,   6.09902333e-06],
        ..., 
        [  7.73450390e-08,   1.25817763e-06],
        [  1.49027598e-07,   1.63649493e-06],
        [  3.98865983e-07,   8.23863736e-07]]),
 array([[  5.49074556e-05,   5.90497159e-03],
        [  3.24894763e-05,   1.60198213e-04],
        [  4.69171772e-05,   2.21179823e-04],
        [  5.42903269e-05,   2.38876730e-04],
        [  3.81234843e-05,   1.53530572e-04],
        [  3.62109259e-05,   1.82565967e-04],
        [  4.17833367e-05,   1.69124509e-04],
        [  6.04446723e-05,   1.98360072e-04],
        [  6.29309557e-05,   2.27295698e-04],
        [  4.82203538e-05,   2.06370416e-04],
        [  4.02543859e-05,   1.70923846e-04],
        [  4.93879561e-05,   1.99861031e-04],
        [  4.14054350e-05,   1.87476124e-04],
        [  4.98337108e-05,   2.23851085e-04],
        [  4.27496542e-05,   1.73524600e-04],
        [  3.53027

In [21]:
print len(tdm.Q), len(tdm.P), len(tdm.docs)

100 128474 100


In [29]:


from matplotlib.pyplot import figure, show
import numpy as npy
from numpy.random import rand

temp = zip(*tdm.P)
x = temp[0]
y = temp[1]
print len(x), len(temp)

if 1: # picking on a scatter plot (matplotlib.collections.RegularPolyCollection)

    
    def onpick3(event):
        ind = event.ind
        #print ind, type(ind)
        if type(ind) == numpy.ndarray:
            #print "YOU"
            ind = ind[0]
        print 'onpick3 scatter:', tdm.terms[ind]#, npy.take(x, ind), npy.take(y, ind)

    fig = figure()
    ax1 = fig.add_subplot(111)
    col = ax1.scatter(x, y, picker=True)
    #fig.savefig('pscoll.eps')
    fig.canvas.mpl_connect('pick_event', onpick3)



128474 2


In [30]:
show()

onpick3 scatter: |
onpick3 scatter: 
onpick3 scatter: =
onpick3 scatter: es
onpick3 scatter: wie
onpick3 scatter: wie
onpick3 scatter: der
onpick3 scatter: das
onpick3 scatter: mich
onpick3 scatter: er
onpick3 scatter: ich
onpick3 scatter: und
onpick3 scatter: 
onpick3 scatter: du
onpick3 scatter: ja
onpick3 scatter: ja
onpick3 scatter: meinen
onpick3 scatter: meinen
onpick3 scatter: —
onpick3 scatter: gases
onpick3 scatter: stack
onpick3 scatter: willst
onpick3 scatter: gut,
onpick3 scatter: es,
onpick3 scatter: loch
onpick3 scatter: ruck
onpick3 scatter: ton
onpick3 scatter: in
onpick3 scatter: _per
onpick3 scatter: an
onpick3 scatter: ah
onpick3 scatter: dicker
onpick3 scatter: rosa
onpick3 scatter: x
onpick3 scatter: x
onpick3 scatter: 2
onpick3 scatter: y
onpick3 scatter: log
onpick3 scatter: log
onpick3 scatter: o
onpick3 scatter: log
onpick3 scatter: log
onpick3 scatter: [
onpick3 scatter: rosa.
onpick3 scatter: leopold.
onpick3 scatter: stand.
onpick3 scatter: stand.
onpick3 sc

In [ ]:
# Aaaand now I can't undo inline plotting
# I stand by matplotlib is a pain
# In other news I'm waiting on spectral embedding to finish 
# in the command line because I guess sleep isn't needed

#soo.. I'll do it hsere too
k=2
n_neighbors=20
from sklearn import manifold
se = manifold.SpectralEmbedding(n_components=k, n_neighbors=n_neighbors)
Yse = se.fit_transform(tdm.tdm)

# Run this in your terminal, not here. It isn't optimized and takes about.. well at least 6x longer. 
#   I'll maybe update that statement when it completes 

In [33]:
import kmmessage
print len(Yse)

100


In [ ]:
kmmessage.sms_message_Send("", "Stuffs done Yo")

In [ ]:

temp = zip(*Yse)
x = temp[0]
y = temp[1]
print len(x), len(temp)

if 1: # picking on a scatter plot (matplotlib.collections.RegularPolyCollection)

    
    def onpick3(event):
        ind = event.ind
        print ind, type(ind)
        if type(ind) == numpy.ndarray:
            print "YOU"
            ind = ind[0]
        print 'onpick3 scatter:', tdm.docs[ind]#, npy.take(x, ind), npy.take(y, ind)

    fig = figure()
    ax1 = fig.add_subplot(111)
    col = ax1.scatter(x, y, picker=True)
    #fig.savefig('pscoll.eps')
    fig.canvas.mpl_connect('pick_event', onpick3)
show()


Dear lord this is why people like linear
Get your act together SE, you're pretty much my 
favorite NLDR method and you're being a total pain.

Though, to be fair, SE works a whole hell of a lot better if the high dimensional space has a lot of coverage. From looking at our test sets with 

In [2]:

sys.executable

'/home/keith/anaconda2/bin/python'